# Connect to Power BI service
Login with Azure Active Directory user authentication.

<b>Note:<b> Service Principal credentials are currently not supported.

In [ ]:
Connect-PowerBIServiceAccount

# Output

In [1]:
$outPath = "C:\Power BI\Activity Logs\"

The below script needs to be able to find the current output cell; this is an easy method to get it.

# Historical Events Offset

Refer to your organizations default audit log retention policy for maximum number of days stored. Default is generally a 90 day period.

Additional Information: https://docs.microsoft.com/en-us/microsoft-365/compliance/audit-log-retention-policies?view=o365-worldwide

In [2]:
$offsetDays = 89

# Error Handling: 

Determines if the variable outPath's final character is a forward slash and that folder exists.

In [3]:
if ($outPath.Substring($outPath.Length - 1, 1) -cne "\") { $outPath = $outPath + "\" }
if (!(Test-Path $outPath)) { New-Item -ItemType Directory -Force -Path $outPath }

# Iterate Offset Date Range

In [ ]:
For ($i = 1; $i -le $offsetDays; $i+=1) { 
    $startEvent = ((Get-Date).AddDays(-$i).ToString("yyyy-MM-ddT00:00:00.000"))
    $endEvent = ((Get-Date).AddDays(-$i).ToString("yyyy-MM-ddT23:59:59.999"))
    
    Write-Host "Evaluating Activity Log: $($startEvent.Substring(0,10))"
    $pbiActivities = Get-PowerBIActivityEvent -StartDateTime $startEvent -EndDateTime $endEvent | ConvertFrom-Json

    if ($pbiActivities.count -ne 0) {
        $outDate = $startEvent.Substring(0,10) -replace '-',''
        $pbiActivities | Export-Csv -Path "$($outpath)Power_BI_Activity_Logs_$($outDate).csv" -NoTypeInformation -Force -Append
    }
}

# Disconnect from Power BI service

In [ ]:
Disconnect-PowerBIServiceAccount